In [36]:
import pandas as pd
from statsapi import player_stats
from pybaseball import playerid_lookup, batting_stats_range, pitching_stats_range
from datetime import datetime
import csv

In [57]:
start_date = '2021-01-01'
start_season = 2021

today = datetime.now()
end_date = today.strftime('%Y-%m-%d')

season = today.year

In [54]:
batting = batting_stats_range(start_date, end_date)
pitching = pitching_stats_range(start_date, end_date)


In [55]:
batter_ids = batting[['Name', 'Tm', 'mlbID']]
pitching_ids = pitching[['Name', 'Tm', 'mlbID']]

In [56]:
batter_ids.to_csv('batter_ids.csv')
pitching_ids.to_csv('pitching_ids.csv')